<a href="https://colab.research.google.com/github/ChairaWandau/MLHomeworks/blob/main/ML_LW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторная работа №3
Студентка Самсонова Валерия, группа М8О-107М-22

Выполнить реализацию GD в Numba и сравнить с реализацией в ЛР №1

##Часть 1

Подключаем необходимые библиотеки

In [1]:
import numpy as np
import numba as nb
from numba import jit
from datetime import datetime
from typing import Tuple, Mapping

Записываем функцию Гольдшейна-Прайса и ее частные производные

In [4]:
#Функция Гольдшейна-Прайса
def GP_E(x,y):
  return (1+(x+y+1)**2 * (19-14*x+3*x**2-14*y+6*x*y+3*y**2))*(30+(2*x-3*y)**2 * (18-32*x+12*x**2+48*y-36*x*y+27*y**2))
#Производная по x
def GP_dEdx(x,y):
  return 24*(8*x**3-4*x**2*(9*y-4)+6*x*(9*y**2+8*y+1)-9*y*(3*y**2+4*y+1))*((3*x**2+2*x*(3*y-7)+3*y**2-14*y+19)*(x+y+1)**2+1)+12*(x**3+x**2*(3*y-2)+x*(3*y**2-4*y-1)+y**3-2*y**2-y+2)*((12*x**2-4*x*(9*y+8)+3*(9*y**2+16*y+6))*(2*x-3*y)**2+30)
#Производная по y
def GP_dEdy(x,y):
  return 12*(x**3+x**2*(3*y-2)+x*(3*y**2-4*y-1)+y**3-2*y**2-y+2)*((12*x**2-4*x*(9*y+8)+3*(9*y**2+16*y+6))*(2*x-3*y)**2+30)-36*(8*x**3-4*x**2*(9*y+4)+6*x*(9*y**2+8*y+1)-9*y*(3*y**2+4*y+1))*((3*x**2+2*x*(3*y-7)+3*y**2-14*y+19)*(x+y+1)**2+1)

Записываем функцию Химмельблау и ее частные производные

In [5]:
#Функция Химмельблау
def H_E(x,y):
  return (x**2+y-11)**2+(x+y**2-7)**2
#Производная по x
def H_dEdx(x,y):
  return 2*(2*x*(x**2+y-11)+x+y**2-7)
#Производная по y
def H_dEdy(x,y):
  return 2*(x**2+2*y*(x+y**2-7)+y-11)

Метод градиентного спуска из ЛР №1

In [6]:
def GradientDescent(E, dEdx, dEdy, x0, y0, LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x0, y0 - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  #Алгоритм градиентного спуска
  x1=x0-LearningRate * dEdx(x0,y0)
  y1=y0-LearningRate * dEdy(x0,y0)
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(x0,y0)
    y1=y0-LearningRate * dEdy(x0,y0)
  return x1,y1, E(x1,y1)

Тест для функции Гольдшейна-Прайса

In [39]:
start_time_without_NUMBA_GP = datetime.now()
x, y, GPStopPoint = GradientDescent(GP_E, GP_dEdx, GP_dEdy, -0.2, 0.2, 0.0001, 500000)
time_without_NUMBA_GP = datetime.now() - start_time_without_NUMBA_GP
print("Время работы без NUMBA\n", time_without_NUMBA_GP)
print('x={0}, y={1}, E={2}'.format(x,y,GPStopPoint))

Время работы без NUMBA
 0:00:02.195232
x=-5.739370960047311e-16, y=-1.0000000000000013, E=2.999999999999986


Тест для функции Химмельблау

In [40]:
start_time_without_NUMBA_H = datetime.now()
x,y,HStopPoint = GradientDescent(H_E, H_dEdx, H_dEdy, -5, 5, 0.001, 500000)
time_without_NUMBA_H = datetime.now() - start_time_without_NUMBA_H
print("Время работы без NUMBA\n", time_without_NUMBA_H)
print('x={0}, y={1}, E={2}'.format(x,y,HStopPoint))

Время работы без NUMBA
 0:00:00.301633
x=-2.805118086952748, y=3.1313125182505757, E=5.837570698635487e-28


##Часть 2
Реализация градиентного спуска через NUMBA

Записываем функцию Гольдшейна-Прайса и ее частные производные

In [9]:
#Функция Гольдшейна-Прайса
@nb.njit(fastmath=True)
def NB_GP_E(x: np.ndarray) -> np.float64:
  return (1+(x[0]+x[1]+1)**2 * (19-14*x[0]+3*x[0]**2-14*x[1]+6*x[0]*x[1]+3*x[1]**2))*(30+(2*x[0]-3*x[1])**2 * (18-32*x[0]+12*x[0]**2+48*x[1]-36*x[0]*x[1]+27*x[1]**2))
#Производная по x
@nb.njit(fastmath=True)
def NB_GP_dEdx(x: np.ndarray) -> np.float64:
  return 24*(8*x[0]**3-4*x[0]**2*(9*x[1]-4)+6*x[0]*(9*x[1]**2+8*x[1]+1)-9*x[1]*(3*x[1]**2+4*x[1]+1))*((3*x[0]**2+2*x[0]*(3*x[1]-7)+3*x[1]**2-14*x[1]+19)*(x[0]+x[1]+1)**2+1)+12*(x[0]**3+x[0]**2*(3*x[1]-2)+x[0]*(3*x[1]**2-4*x[1]-1)+x[1]**3-2*x[1]**2-x[1]+2)*((12*x[0]**2-4*x[0]*(9*x[1]+8)+3*(9*x[1]**2+16*x[1]+6))*(2*x[0]-3*x[1])**2+30)
#Производная по y
@nb.njit(fastmath=True)
def NB_GP_dEdy(x: np.ndarray) -> np.float64:
  return 12*(x[0]**3+x[0]**2*(3*x[1]-2)+x[0]*(3*x[1]**2-4*x[1]-1)+x[1]**3-2*x[1]**2-x[1]+2)*((12*x[0]**2-4*x[0]*(9*x[1]+8)+3*(9*x[1]**2+16*x[1]+6))*(2*x[0]-3*x[1])**2+30)-36*(8*x[0]**3-4*x[0]**2*(9*x[1]+4)+6*x[0]*(9*x[1]**2+8*x[1]+1)-9*x[1]*(3*x[1]**2+4*x[1]+1))*((3*x[0]**2+2*x[0]*(3*x[1]-7)+3*x[1]**2-14*x[1]+19)*(x[0]+x[1]+1)**2+1)

Записываем функцию Химмельблау и ее частные производные

In [10]:
#Функция Химмельблау
@nb.njit(fastmath=True)
def NB_H_E(x: np.ndarray):
  return (x[0]**2+x[1]-11)**2+(x[0]+x[1]**2-7)**2
#Производная по x
@nb.njit(fastmath=True)
def NB_H_dEdx(x: np.ndarray):
  return 2*(2*x[0]*(x[0]**2+x[1]-11)+x[0]+x[1]**2-7)
#Производная по y
@nb.njit(fastmath=True)
def NB_H_dEdy(x: np.ndarray):
  return 2*(x[0]**2+2*x[1]*(x[0]+x[1]**2-7)+x[1]-11)

Метод градиентного спуска c NUMBA

In [11]:
@nb.njit(fastmath=True)
def NB_GradientDescent(E: Mapping, dEdx: Mapping, dEdy: Mapping, x:np.ndarray,LearningRate, Iterations):
  '''
  E - функция
  dEdx, dEdy - частные производные функции
  x - начальные приближения точки
  LearningRate - скорость обучения
  Iterations - количество итераций
  '''
  #Алгоритм градиентного спуска
  x0 = x[0];
  y0 = x[1];
  x1=x0-LearningRate * dEdx(np.array([x0, y0]))
  y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  for _ in range(Iterations):
    x0=x1
    y0=y1
    x1=x0-LearningRate * dEdx(np.array([x0, y0]))
    y1=y0-LearningRate * dEdy(np.array([x0, y0]))
  return x1,y1, E(np.array([x1,y1]))

Тест для функции Гольдшейна-Прайса

In [42]:
start_time_with_NUMBA_GP = datetime.now()
x,y,NB_GPStopPoint = NB_GradientDescent(NB_GP_E, NB_GP_dEdx, NB_GP_dEdy, np.array([-0.2, 0.2]), 0.0001, 50000)
time_with_NUMBA_GP = datetime.now() - start_time_with_NUMBA_GP
print("Время работы с NUMBA:\n", time_with_NUMBA_GP)
print('x={0}, y={1}, E={2}'.format(x,y,NB_GPStopPoint))

Время работы с NUMBA:
 0:00:00.006462
x=-5.551115123125774e-16, y=-1.0000000000000013, E=2.9999999999999813


Тест для функции Химмельблау

In [43]:
start_time_with_NUMBA_H = datetime.now()
x,y,NB_HStopPoint = NB_GradientDescent(NB_H_E, NB_H_dEdx, NB_H_dEdy, np.array([-5,5]), 0.001, 500000)
time_with_NUMBA_H = datetime.now() - start_time_with_NUMBA_H
print("Время работы с NUMBA:\n", time_with_NUMBA_H)
print('x={0}, y={1}, E={2}'.format(x,y,NB_HStopPoint))

Время работы с NUMBA:
 0:00:00.048319
x=-2.805118086952748, y=3.1313125182505757, E=6.11339851131986e-28


Сравнение времени работы программы без NUMBA и с NUMBA

In [44]:
print("Время работы программы для функции Гольдшейна-Прайса без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_GP/time_with_NUMBA_GP)))
print("Время работы программы для функции Химмельблау без NUMBA больше в {0} раз, по сравнению со временем работы с NUMBA.".format(round(time_without_NUMBA_H/time_with_NUMBA_H)))

Время работы программы для функции Гольдшейна-Прайса без NUMBA больше в 340 раз, по сравнению со временем работы с NUMBA.
Время работы программы для функции Химмельблау без NUMBA больше в 6 раз, по сравнению со временем работы с NUMBA.
